In [1]:
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path='conf/')

/tmp/ipykernel_3249328/2329082962.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='conf/')


hydra.initialize()

In [2]:
cfg = compose(config_name='config')

import tensorflow as tf
import math
import pandas as pd
import numpy as np

from pathlib import Path

from src import datasets 

2024-07-25 10:15:08.220303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 10:15:08.242296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 10:15:08.242323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 10:15:08.257114: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 10:15:10.003529: W tensorflow/com

In [3]:
# Load the data
uni_cfg = cfg.dataset
dat_cfg = cfg.dataset.args.test
labels = pd.read_csv(cfg.dataset.labels)
labels_len = len(labels)
batch_size = cfg.mode.batch_size

del labels

dataset = datasets.WayneCroppedDataset(
    data_idx = range(labels_len),
    shuffle=dat_cfg.shuffle,
    balance=dat_cfg.balance,
    batch_size=batch_size,
    data_dir=uni_cfg.data_dir,
    labels = uni_cfg.labels,
    channels = uni_cfg.channels
)

assert int(math.ceil(labels_len / batch_size)) == len(dataset)

In [4]:
# Load model
model_path = '/work/users/o/e/oem/PHASER/saved_models/alexnet_defaults/2024-07-23/epoch_6_vloss_0.15.h5'
model_path = Path(model_path)

model = tf.keras.models.load_model(model_path)

2024-07-25 10:15:29.432413: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14784 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:1a:00.0, compute capability: 7.0


In [5]:
colnames = ['cell_id', 'prediction', 'confidence', 'true_lab']
phase_labels = ['G1', 'S', 'G2']
results_df = pd.DataFrame(columns=colnames)

for s, step in enumerate(dataset):
    
    if len(step[0]) == 0:
        break
    
    predictions = model.predict(step[0])
    confidence = np.max(predictions, axis=-1)
    predictions = np.argmax(predictions, axis=-1)
    predictions = np.array([phase_labels[i] for i in predictions])
    
    true_lab = step[1].numpy()
    true_lab = np.argmax(true_lab, axis=-1)
    true_lab = np.array([phase_labels[i] for i in true_lab])
    
    cell_ids = step[-1].numpy()
    
    stacked = np.vstack([cell_ids, predictions, confidence, true_lab])
    
    temp_df = pd.DataFrame(stacked.T, columns=colnames)
    
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

I0000 00:00:1721916936.526390 3249357 service.cc:145] XLA service 0x7f0690005c70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721916936.526419 3249357 service.cc:153]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-07-25 10:15:36.533696: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-25 10:15:36.587005: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 689ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


I0000 00:00:1721916937.111769 3249357 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

In [6]:
results_df['cell_id'] = results_df['cell_id'].str.decode('utf-8')

results_df

,cell_id,prediction,confidence,true_lab
0,cell_4415,S,0.940694,S
1,cell_8147,G1,0.988647,G1
2,cell_3629,S,0.999434,S
3,cell_1894,G1,0.998362,G1
4,cell_5900,G1,0.993839,G1
...,...,...,...,...
5870,cell_3227,S,0.999781,S
5871,cell_0965,G1,0.999068,G1
5872,cell_1247,G1,0.999089,G1
5873,cell_1647,G1,0.999009,G1


In [7]:
# Save the results
save_results = '/work/users/o/e/oem/PHASER/results/inference_on_all_cells.csv'
save_results = Path(save_results)
save_results.parent.mkdir(exist_ok=True, parents=True)
results_df.to_csv(save_results)